In [ ]:
#coding=utf-8

'''
Enhancer Transformer的实现
'''

from numpy.core.fromnumeric import var
import torch
from torch.nn.modules.loss import BCELoss
from torch.utils.data.dataloader import DataLoader
from torch import nn
import torch.nn.functional as F
import math
from torch.utils.data import Dataset, Subset
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from torch import Tensor

In [ ]:
class TransformerEmbedding(nn.Module):
    def __init__(self, embedding_num, embedding_dim, pad_idx, device) -> None:
        super().__init__()
        self.embedding = nn.Embedding(embedding_num, embedding_dim, pad_idx, device=device)

    def forward(self, x):
        x = self.embedding(x)
        return x


In [ ]:
# class PositionalEncoding(nn.Module):
#     """
#     compute sinusoid encoding.
#     """
#     def __init__(self, d_model, max_len, device):
#         """
#         constructor of sinusoid encoding class

#         :param d_model: dimension of model
#         :param max_len: max sequence length
#         :param device: hardware device setting
#         """
#         super(PositionalEncoding, self).__init__()

#         # same size with input matrix (for adding with input matrix)
#         self.encoding = torch.zeros(max_len, d_model, device=device)
#         self.encoding.requires_grad = False  # we don't need to compute gradient

#         pos = torch.arange(0, max_len, device=device)
#         pos = pos.float().unsqueeze(dim=1)
#         # 1D => 2D unsqueeze to represent word's position

#         _2i = torch.arange(0, d_model, step=2, device=device).float()
#         # 'i' means index of d_model (e.g. embedding size = 50, 'i' = [0,50])
#         # "step=2" means 'i' multiplied with two (same with 2 * i)
#         self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
#         if d_model % 2 == 1:
#             _2i = _2i[:-1]
#         self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))
#         # compute positional encoding to consider positional information of words

#     def forward(self, x):
#         # self.encoding
#         # [max_len = 512, d_model = 512]

#         batch_size, seq_len, d_model= x.size()
#         # [batch_size = 128, seq_len = 30]

#         return x + self.encoding[:seq_len, :]
#         # [seq_len = 30, d_model = 512]
#         # it will add with tok_emb : [128, 30, 512]



In [ ]:
# 以突变位点为中心实现位置编码

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, seq_len, device) -> None:
        super().__init__()
        self.encoding = torch.zeros(seq_len, d_model, device=device)
        self.encoding.requires_grad = False
    
        assert seq_len%2==1
        pos = torch.arange(0, seq_len, device=device)
        half = int((seq_len-1)/2)
        pos[:half] = torch.arange(half, 0, step=-1)
        pos[half+1:] = torch.arange(1, half+1)
        pos[half] = 0
        pos = pos.float().unsqueeze(dim=1)

        _2i = torch.arange(-0, d_model, step=2, device=device).float()
        self.encoding[:, 0::2] = torch.sin(pos/(10**(_2i/d_model)))
        if d_model%2 == 1:
            _2i = _2i[:-1]
        self.encoding[:, 1::2] = torch.cos(pos/(10**(_2i/d_model)))
    
    def getEncoding(self):
        return self.encoding
    
    def forward(self, x):
        batch_size, seq_len, d_model = x.size()
        return x + self.encoding[:seq_len, :]

# p = PositionalEncoding(10, 501, 'cpu')
# fig = plt.figure(figsize=(20, 20))
# ax = fig.add_subplot(111)
# encoding = p.getEncoding()
# print(encoding)
# im=ax.imshow(encoding,cmap='plasma_r')
# plt.show()



In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_concat = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        # 1. dot product with weight matrices
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 2. split tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q, k, v, mask=mask)
        
        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)

        # 5. visualize attention map
        # TODO : we should implement visualization

        return out

    def split(self, tensor):
        """
        split tensor by number of head

        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()

        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, self.n_head, length, d_tensor)
        # it is similar with group convolution (split by number of heads)

        return tensor

    def concat(self, tensor):
        """
        inverse function of self.split(tensor : torch.Tensor)

        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.view(batch_size, length, d_model)
        return tensor



In [ ]:
class ScaleDotProductAttention(nn.Module):
    """
    compute scale dot product attention

    Query : given sentence that we focused on (decoder)
    Key : every sentence to check relationship with Qeury(encoder)
    Value : every sentence same with Key (encoder)
    """

    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(-1)

    def forward(self, q, k, v, mask=None, e=1e-12):
        # input is 4 dimension tensor
        # [batch_size, head, length, d_tensor]
        batch_size, head, length, d_tensor = k.size()

        # 1. dot product Query with Key^T to compute similarity
        # k_t = k.view(batch_size, head, d_tensor, length)  # transpose
        k_t = k.permute(0, 1, 3, 2)
        score = (q @ k_t) / math.sqrt(d_tensor)  # scaled dot product

        # 2. apply masking (opt)
        if mask is not None:
            score = score.masked_fill(mask == 0, -e)

        # 3. pass them softmax to make [0, 1] range
        score = self.softmax(score)

        # 4. multiply with Value
        v = score @ v

        return v, score



In [ ]:
class Attention(nn.Module):

    def __init__(self, seq_len, embedding_dim, device=None) -> None:
        super().__init__()
        self.seq_len = seq_len
        self.device = device
        self.embedding_dim = embedding_dim
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)
        self.layer_norm = nn.LayerNorm([seq_len, 1])
        # self.softmax = nn.Softmax(-1)

    def forward(self, x):
        batch_size, seq_len, d_model = x.size()
        q = self.w_q(x)
        v = self.w_v(x)
        # k = x[:, (seq_len+1)/2, :]
        k = torch.index_select(x, 1, torch.tensor(seq_len//2).to(self.device))
        k_t = k.permute(0, 2, 1)
        # 计算注意力
        score = (q @ k_t) / math.sqrt(d_model)
        # v = score@v
        score = self.layer_norm(score)
        return score

In [ ]:
class Classifier_with_simple_attention(nn.Module):

    def __init__(self, input_dim, output_dim) -> None:
        super().__init__()
        self.linear1 = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU()
        )
        self.linear2 = nn.Sequential(
            nn.Linear(64, output_dim),
            nn.ReLU()
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
class VTC_with_simple_attention(nn.Module):

    def __init__(self, seq_len, embedding_dim, input_dim, output_dim, device=None) -> None:
        super().__init__()
        self.att_layer = Attention(seq_len, embedding_dim, device)
        self.classifier = Classifier_with_simple_attention(input_dim, output_dim)
        self.device = device
    
    def embedding(self, x, y, device):
        batch_size, length = x.size()
        x_ = torch.zeros(batch_size, length, 5).to(device)
        x = x.view(batch_size, -1, 1)
        x_.scatter_(2, x, 1)
        y_ = torch.zeros(batch_size, length, 5).to(device)
        y = y.view(batch_size, -1, 1)
        y_.scatter_(2, y, 1)
        return torch.cat((x_, y_), 2)
    
    def forward(self, x, y):
        x = self.embedding(x, y, self.device)
        x = self.att_layer(x)
        batch_size, seq_len, _ = x.size()
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

In [ ]:
dummy_input1 = torch.randn(10, 2001, 10)
att = Attention(2001, 10)
output = att(dummy_input1)
# dummy_input2 = torch.randn(10, 1001, 5)
# att = VTC_with_simple_attention(1001, 10, 1001, 1)
# output = att(dummy_input1, dummy_input2)

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        # '-1' means last dimension. 

        out = (x - mean) / (std + self.eps)
        out = self.gamma * out + self.beta
        return out



In [ ]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x



In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, n_head=n_head)
        self.norm1 = LayerNorm(d_model=d_model)
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNorm(d_model=d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, src_mask):
        # 1. compute self attention
        _x = x
        x = self.attention(q=x, k=x, v=x, mask=src_mask)
        
        # 2. add and norm
        x = self.norm1(x + _x)
        x = self.dropout1(x)
        
        # 3. positionwise feed forward network
        _x = x
        x = self.ffn(x)
      
        # 4. add and norm
        x = self.norm2(x + _x)
        x = self.dropout2(x)
        return x



In [ ]:
class Encoder(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, n_layers, drop_prob, device):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model=d_model,
                                                  ffn_hidden=ffn_hidden,
                                                  n_head=n_head,
                                                  drop_prob=drop_prob)
                                     for _ in range(n_layers)])

    def forward(self, x, src_mask):
        for layer in self.layers:
            x = layer(x, src_mask)

        return x



In [ ]:
class Conv(nn.Module):
    
    def __init__(self, seq_len, embedding_dim, output_dim, kernel_size) -> None:
        super().__init__()
        self.conv = nn.Conv1d(embedding_dim, output_dim, kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        self.max_pool = nn.MaxPool1d(3, 2)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.max_pool(x)
        return x

In [ ]:
"""
use CNN to compress lone sequences
"""

class Convs(nn.Module):
    
    def __init__(self, seq_len, embedding_dim, output_dim, n_layers=4, kernel_size=5) -> None:
        super().__init__()
        self.layers = nn.ModuleList([Conv(seq_len, embedding_dim, output_dim, kernel_size)
            for _ in range(n_layers)
        ])
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

'''
50*5*120 50*5*120
'''

In [ ]:
# dummy_input = torch.rand(50, 2001, 5)
# dummy_input = dummy_input.permute(0, 2, 1)
# convs1 = Convs(2001, 5, 5)
# output = convs1(dummy_input)
# pass

In [ ]:
# class ClassificationLayer(nn.Module):
#     def __init__(self, embedding_dim, seq_len, output_dim) -> None:
#         super().__init__()
#         self.linear1 = nn.Linear(embedding_dim, 1)
#         self.linear2 = nn.Linear(seq_len, output_dim)
#         self.leaky_relu = nn.LeakyReLU()
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         batch_size, _, _ = x.size() 
#         x = self.linear1(x)
#         x = self.leaky_relu(x)
#         x = x.view(batch_size, -1)
#         x = self.linear2(x)
#         x = self.sigmoid(x)
#         return x



In [ ]:
class ClassificationLayer(nn.Module):
    def __init__(self,  embedding_dim, seq_len, output_dim) -> None:
        super().__init__()
        # self.conv1 = nn.Conv1d(10, 2, kernel_size=10)
        # self.relu1 = nn.ReLU()
        # self.conv2 = nn.Conv1d(2, 10, kernel_size=5)
        # self.relu2 = nn.ReLU()
        # self.dropout = nn.Dropout()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=10, out_channels=20, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(3, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels=20, out_channels=40, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(3,2)
        )
        """
        1001 9920
        201 1920
        """
        # 1001 9920
        self.linear1 = nn.Linear(1080, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 1)

        # self.dense = nn.Sequential(
        #     nn.Linear(1080, 512),
        #     nn.ReLU(),
        #     nn.Dropout(),
        #     nn.Linear(512, 256),
        #     nn.ReLU(),
        #     nn.Dropout(),
        #     nn.Linear(256, 1)
        # )
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        self.layernorm1 = nn.LayerNorm(512)
        self.layernorm2 = nn.LayerNorm(256)
        # self.linear1 = nn.Linear(, 50)
    
    def forward(self, x):
        # print("CNN classifier")
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        # x = self.dense(x)
        x = self.linear1(x)
        x = self.dropout(self.relu(x))
        x = self.layernorm1(x)
        x = self.linear2(x)
        x = self.dropout(self.relu(x))
        x = self.layernorm2(x)
        x = self.linear3(x)
        # x = self.dropout(self.relu(x))
        x = self.sigmoid(x)
        return x
        

In [ ]:
# input = torch.randn([50, 10, 201])
# classifier = ClassificationLayer(10, 201, 1)
# out = classifier(input)

In [ ]:
class VariantPathogenicityClassifier(nn.Module):
    def __init__(self, params, device) -> None:
        super().__init__()
        self.pad_idx = params.pad_idx
        self.device = device
        self.positionalEncoding = PositionalEncoding(
            d_model=params.embedding_dim,
            seq_len=params.seq_len,
            device=device
        )
        self.convs = Convs(
            seq_len=params.seq_len,
            embedding_dim=params.embedding_dim,
            output_dim=params.pre_cnn_output_dim,
            n_layers=params.pre_cnn_n_layers)
        self.encoder = Encoder(
            d_model=params.pre_cnn_output_dim,
            ffn_hidden=params.hidden_dim,
            n_head=params.att_head_num,
            n_layers=params.n_layers,
            drop_prob=params.trans_drop_prob,
            device=device
        )
        self.classifier = ClassificationLayer(params.pre_cnn_output_dim, params.pre_cnn_output_seq_len, params.output_dim)
    
    def make_pad_mask(self, q, k, src_padding_idx):
        len_q, len_k = q.size(1), k.size(1)

        # batch_size x 1 x 1 x len_k
        k = k.ne(src_padding_idx).unsqueeze(1).unsqueeze(2)
        # batch_size x 1 x len_q x len_k
        k = k.repeat(1, 1, len_q, 1)

        # batch_size x 1 x len_q x 1
        q = q.ne(src_padding_idx).unsqueeze(1).unsqueeze(3)
        # batch_size x 1 x len_q x len_k
        q = q.repeat(1, 1, 1, len_k)

        mask = k & q
        return mask
    
    def embedding(self, x, y, device):
        batch_size, length = x.size()
        x_ = torch.zeros(batch_size, length, 5).to(device)
        x = x.view(batch_size, -1, 1)
        x_.scatter_(2, x, 1)
        y_ = torch.zeros(batch_size, length, 5).to(device)
        y = y.view(batch_size, -1, 1)
        y_.scatter_(2, y, 1)
        return torch.cat((x_, y_), 2)
          
    def forward(self, x, y):
        # src_mask = self.make_pad_mask(x, x, self.pad_idx)
        src_mask = None
        x = self.embedding(x, y, self.device)
        x = self.positionalEncoding(x)
        x = x.permute(0, 2, 1)
        x = self.convs(x)
        x = x.permute(0, 2, 1)
        x = self.encoder(x, src_mask)
        x = x.permute(0, 2, 1)
        x = self.classifier(x)
        return x



In [ ]:
class MLPClassifier(nn.Module):

    def __init__(self, seq_len, embedding_dim, output_dim, device) -> None:
        super().__init__()
        self.device = device
        self.linear1 = nn.Linear(embedding_dim, 1)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(seq_len, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def embedding(self, x, y, device):
        batch_size, length = x.size()
        x_ = torch.zeros(batch_size, length, 5).to(device)
        x = x.view(batch_size, -1, 1)
        x_.scatter_(2, x, 1)
        y_ = torch.zeros(batch_size, length, 5).to(device)
        y = y.view(batch_size, -1, 1)
        y_.scatter_(2, y, 1)
        return torch.cat((x_, y_), 2)

    def forward(self, x, y):
        x = self.embedding(x, y, self.device)
        x = self.linear1(x)
        x = self.relu(x)
        batch_size, _, _ = x.size() 
        x = x.view(batch_size, -1)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
# 自定义加权loss
class WeightedBCELoss(nn.Module):

    def __init__(self, pos_weight: float=1.0,  reduction: str='mean') -> None:
        super().__init__()
        self.pos_weight = pos_weight
        self.reduction = reduction
    
    def forward(self, input:Tensor, target:Tensor) -> Tensor:
        loss = F.binary_cross_entropy(input, target, reduction='none')
        weights = torch.full_like(target, 1-self.pos_weight)
        weights[target==1] = self.pos_weight
        loss = loss * weights
        if self.reduction == 'mean':
            loss = torch.mean(loss)
        elif self.reduction == 'sum':
            loss = torch.sum(loss)

        return loss
        
        

In [ ]:
# t = torch.randn(50, 1)
# input = torch.rand_like(t)
# target = torch.randint_like(t, 2)
# print(input)
# print(target)
# loss_func = WeightedBCELoss()
# pos_weight = 0.75
# loss = loss_func(input, target, pos_weight)
# print(loss)

In [ ]:
import imp
params = imp.load_source('params', 'params.py')

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
class VariantDataset(Dataset):
    def __init__(self, file_path) -> None:
        self.raw_data = pd.read_csv(file_path)
        self.encode_seq()

    def __len__(self):
        return self.raw_data.shape[0]

    def __getitem__(self, index):
        variant = self.raw_data.iloc[index]
        ref = torch.tensor(variant['encoded_ref'])
        alt = torch.tensor(variant['encoded_alt'])
        label = torch.tensor(variant['label'])
        return [ref, alt, label]
    
    def encode_seq(self):
        # 编码
        dict_ = {
            'A': 1,
            'T': 2,
            'C': 3,
            'G': 4,
            'N': 0
        }
        self.raw_data['encoded_ref'] = self.raw_data['seq_1000_ref'].apply(lambda x: list(map(lambda i: dict_[i], list(x))))
        self.raw_data['encoded_alt'] = self.raw_data['seq_1000_alt'].apply(lambda x: list(map(lambda i: dict_[i], list(x))))
                

In [ ]:
trainset = VariantDataset('../data/train.csv')
validset = VariantDataset('../data/valid.csv')

In [ ]:
trainLoader = DataLoader(trainset, batch_size=params.batch_size, shuffle=True)
validLoader = DataLoader(validset, batch_size=params.batch_size, shuffle=True)

In [ ]:
# model = VariantPathogenicityClassifier(params, device)

# model = MLPClassifier(
#     seq_len=seq_len,
#     embedding_dim = embedding_dim,
#     output_dim=output_dim,
#     device=device
# )

model = VTC_with_simple_attention(params.seq_len, params.embedding_dim, params.seq_len, params.output_dim, device)
model=model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=params.learning_rate)

# loss_func = BCELoss()
loss_func = WeightedBCELoss(pos_weight=0.75)

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    loss_sum = 0
    for batch_idx, (x, y, target) in enumerate(train_loader):
        x, y, target = x.to(device), y.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(x, y)
        target = torch.unsqueeze(target, 1)
        # output=output.to(torch.float32)
        target=target.to(torch.float32)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        if batch_idx % params.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    loss_avg = loss_sum/(batch_idx+1)
    writer.add_scalar('Train_loss', loss_avg, epoch)

In [ ]:
def metrics(confusion_matrix):
    epsilon = 1e-7
    sum = np.sum(confusion_matrix)
    tn, fp, fn, tp = confusion_matrix.ravel()
    accuracy = (tn + tp)/sum
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    f1 = 2*precision*recall/(precision+recall+epsilon)
    return accuracy, precision, recall, f1


In [ ]:
def valid(model, device, validloader, epoch):
    model.eval()
    test_loss = 0
    thresholds = [0.2, 0.25, 0.3, 0.4, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.6, 0.7]
    threshold_num = len(thresholds)
    correct = [0 for i in range(threshold_num)]
    y_pred_thres = [[] for i in range(threshold_num)]
    confusion_matrixs = [np.array([[0, 0], [0, 0]]) for i in range(threshold_num)]
    y_true =[]
    with torch.no_grad():
        for x, y, target in validloader:
            x, y, target = x.to(device), y.to(device), target.to(device)
            output = model(x, y)
            target = torch.unsqueeze(target, 1)
            target = target.to(torch.float32)
            test_loss += F.binary_cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            target = target.reshape(1, -1)
            # y_true.extend(target[0])
            for i in range(len(thresholds)):
                pred = (output > thresholds[i]).to(torch.float32)
                pred = pred.reshape(1, -1)
                # y_pred_thres[i].extend(pred[0])
                matched = pred.eq(target).sum().item()
                # correct[i] = correct[i] + matched
                confusion_matrixs[i] = confusion_matrixs[i] + confusion_matrix(target[0].cpu(), pred[0].cpu())
                
    test_loss /= len(validLoader.dataset)
    writer.add_scalar('Test_loss', test_loss, epoch)
    # acc = []
    # for c in correct:
    #     acc.append(c / (len(validLoader.dataset)))
    # writer.add_scalar('Validation_accuracy', acc[2], epoch)
    accs = {}
    precs = {}
    recalls = {}
    f1s = {}
    for i in range(threshold_num):
        accuracy, precision, recall, f1= metrics(confusion_matrixs[i])
        print(thresholds[i], accuracy, precision, recall, f1)
        accs[str(thresholds[i])] = accuracy
        precs[str(thresholds[i])] = precision
        recalls[str(thresholds[i])] = recall
        f1s[str(thresholds[i])] = f1
    writer.add_scalars('Evaluation_performance_acc', accs, global_step=epoch)
    writer.add_scalars('Evaluation_performance_precision', precs, global_step=epoch)
    writer.add_scalars('Evaluation_performance_recall', recalls, global_step=epoch)
    writer.add_scalars('Evaluation_performance_f1', f1s, global_step=epoch)

    print('\nTest set: Average loss: {:.4f}'.format(test_loss))



In [ ]:
for i in range(params.epoch):
    print(i)
    train(model, device, trainLoader, optimizer, i)
    valid(model, device, validLoader, i)

In [ ]:
valid(model, device, validLoader, 1)